In [1]:
!pip install -q ultralytics roboflow optuna

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="b3Y3YIe0efqyQvtGEtt2")
project = rf.workspace("cchegeu").project("russian-signs")
dataset = project.version(15).download("yolov8")

In [ ]:
import optuna
from ultralytics import YOLO

def objective(trial):
    # Define the hyperparameters to be tuned
    epochs = trial.suggest_int('epochs', 1, 10)
    optimizer_name = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW', 'RMSProp', 'auto'])
    cos_lr = trial.suggest_categorical('cos_lr', [True, False])
    lr0 = trial.suggest_float('lr0', 1e-6, 1e-2)
    label_smoothing = trial.suggest_float('label_smoothing', 0.0, 0.6)
    warmup_momentum = trial.suggest_float('warmup_momentum', 0, 1)

    # Set up YOLO model with the suggested parameters
    model = YOLO("yolov8n.pt")
    model.train(data="Russian-signs-15/data.yaml", epochs=epochs, optimizer=optimizer_name, cos_lr=cos_lr, lr0=lr0, label_smoothing=label_smoothing, warmup_momentum=warmup_momentum)
    return model.val(data="Russian-signs-15/data.yaml",).results_dict["metrics/recall(B)"]

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # Modify `n_trials` as needed

# Log results
experiment_results = {}
for trial in study.trials:
    experiment_results[trial.number] = {
        'params': trial.params,
        'recall': trial.value
    }

# Save or print experiment results
print(experiment_results)

# Further analysis or use of the best parameters
best_params = study.best_params
print("Best parameters:", best_params)

[I 2024-01-13 23:59:38,289] A new study created in memory with name: no-name-58099f85-f974-4dcf-8b1a-d5bc0982f9e1
/home/rsolomatin/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Ultralytics YOLOv8.1.1 🚀 Python-3.8.8 torch-2.1.2+cu121 CPU (AMD Ryzen 7 2700 Eight-Core Processor)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=Russian-signs-15/data.yaml, epochs=9, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=RMSProp, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning /home/rsolomatin/avito/Russian-signs-15/train/labels.cache... 2094 images, 36 backgrounds, 0 corrupt: 100%|██████████| 2094/2094 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/rsolomatin/avito/Russian-signs-15/valid/labels.cache... 596 images, 3 backgrounds, 0 corrupt: 100%|██████████| 596/596 [00:00<?, ?it/s]

...

Plotting labels to runs/detect/train11/labels.jpg... 


optimizer: RMSprop(lr=0.005290821523495658, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


2024/01/14 01:38:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2024/01/14 01:38:26 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2024/01/14 01:38:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2024/01/14 01:38:26 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/01/14 01:38:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of statsmodels. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a supported version, or try upgrading MLflow.
2024/01/14 01:38:26 INFO mlflow.tracking.fluent: Autologging suc

MLflow: logging run_id(b8cead726a214f88a5f1aafe34a27632) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
7 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/7         0G      2.262      3.514      1.319        150        640:  58%|█████▊    | 76/131 [04:49<03:30,  3.82s/it]

In [ ]:
import pickle

file_path = 'optuna_study_results.pkl'  # Path where you want to save the file
with open(file_path, 'wb') as file:
    pickle.dump(experiment_results, file)

print(f"Results saved to {file_path}")